In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                # Sample generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                # Train the critic
                d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                                [valid, fake, dummy])

            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.generator_model.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

            # If at save interval => save generated image samples
            if epoch>=29000:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\WGAN_Improved Images\mnist_%d.png" % epoch)
        plt.close()

In [4]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    wgan = WGANGP()
    wgan.train(epochs=30000, batch_size=32, sample_interval=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_______________________________________

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 11.522637] [G loss: 0.105916]


c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 9.173203] [G loss: 0.152647]
2 [D loss: 8.492870] [G loss: -0.121066]
3 [D loss: 8.873832] [G loss: -0.309383]
4 [D loss: 7.232145] [G loss: 0.134041]
5 [D loss: 6.114133] [G loss: -0.079763]
6 [D loss: 5.713455] [G loss: -0.047343]
7 [D loss: 4.085890] [G loss: 0.071953]
8 [D loss: 3.821262] [G loss: -0.023185]
9 [D loss: 2.782443] [G loss: 0.013409]
10 [D loss: 2.172705] [G loss: -0.144384]
11 [D loss: 1.434293] [G loss: -0.160206]
12 [D loss: 1.861977] [G loss: -0.384587]
13 [D loss: 0.630567] [G loss: -0.237335]
14 [D loss: -0.092834] [G loss: -0.244242]
15 [D loss: -0.443601] [G loss: -0.320402]
16 [D loss: -0.290713] [G loss: -0.332890]
17 [D loss: 0.525502] [G loss: -0.380129]
18 [D loss: -0.176370] [G loss: -0.446295]
19 [D loss: -0.588999] [G loss: -0.587928]
20 [D loss: -0.453906] [G loss: -0.321543]
21 [D loss: -0.952482] [G loss: -0.604689]
22 [D loss: -0.517680] [G loss: -0.548890]
23 [D loss: -0.680441] [G loss: -0.441894]
24 [D loss: -1.088925] [G loss: -0.856

...

29153 [D loss: -0.265595] [G loss: -4.098060]
29154 [D loss: -1.027062] [G loss: -3.229818]
29155 [D loss: -0.730040] [G loss: -4.090174]
29156 [D loss: -0.142270] [G loss: -4.097979]
29157 [D loss: 0.348011] [G loss: -2.978264]
29158 [D loss: -0.145005] [G loss: -4.180820]
29159 [D loss: -0.205068] [G loss: -3.645043]
29160 [D loss: -0.981719] [G loss: -3.624515]
29161 [D loss: -0.598453] [G loss: -4.120244]
29162 [D loss: -0.279126] [G loss: -3.835389]
29163 [D loss: -0.165818] [G loss: -4.340141]
29164 [D loss: -0.830523] [G loss: -3.770368]
29165 [D loss: -0.002492] [G loss: -4.103160]
29166 [D loss: -0.258631] [G loss: -4.111670]
29167 [D loss: 0.492627] [G loss: -4.095343]
29168 [D loss: -0.164760] [G loss: -4.166696]
29169 [D loss: 0.429517] [G loss: -4.046680]
29170 [D loss: 0.124497] [G loss: -3.268712]
29171 [D loss: 0.043599] [G loss: -4.189343]
29172 [D loss: -0.986289] [G loss: -4.714390]
29173 [D loss: 0.389399] [G loss: -4.243519]
29174 [D loss: -0.340669] [G loss: -4.20

29333 [D loss: 0.029587] [G loss: -4.572222]
29334 [D loss: -0.738662] [G loss: -3.951857]
29335 [D loss: -0.385272] [G loss: -4.779990]
29336 [D loss: -0.458316] [G loss: -3.747715]
29337 [D loss: -0.351819] [G loss: -4.260546]
29338 [D loss: -0.386687] [G loss: -4.351343]
29339 [D loss: 0.284685] [G loss: -4.056309]
29340 [D loss: -0.428450] [G loss: -4.118902]
29341 [D loss: 0.740239] [G loss: -3.897460]
29342 [D loss: -0.531617] [G loss: -4.195160]
29343 [D loss: -0.608684] [G loss: -3.474882]
29344 [D loss: 0.289087] [G loss: -4.115908]
29345 [D loss: 0.860604] [G loss: -4.834535]
29346 [D loss: -0.424293] [G loss: -3.802225]
29347 [D loss: -0.058297] [G loss: -5.217672]
29348 [D loss: 0.068892] [G loss: -4.380304]
29349 [D loss: -0.162325] [G loss: -4.589402]
29350 [D loss: -0.876919] [G loss: -4.189685]
29351 [D loss: -0.453320] [G loss: -4.271246]
29352 [D loss: 0.011191] [G loss: -4.879051]
29353 [D loss: -0.792480] [G loss: -3.935555]
29354 [D loss: -0.352613] [G loss: -4.230

29513 [D loss: -0.719481] [G loss: -4.260323]
29514 [D loss: -0.484829] [G loss: -3.981178]
29515 [D loss: -0.026417] [G loss: -4.393370]
29516 [D loss: -0.425100] [G loss: -4.855648]
29517 [D loss: -0.489915] [G loss: -4.196270]
29518 [D loss: 0.182461] [G loss: -4.051383]
29519 [D loss: -0.015959] [G loss: -3.979686]
29520 [D loss: -0.219582] [G loss: -4.365604]
29521 [D loss: -0.881838] [G loss: -3.966974]
29522 [D loss: -0.238904] [G loss: -4.415396]
29523 [D loss: 0.297998] [G loss: -4.598552]
29524 [D loss: -0.238412] [G loss: -4.896936]
29525 [D loss: -0.523378] [G loss: -4.186637]
29526 [D loss: -0.308610] [G loss: -4.138260]
29527 [D loss: -0.466719] [G loss: -4.085023]
29528 [D loss: 0.359420] [G loss: -4.728852]
29529 [D loss: -0.771703] [G loss: -4.123189]
29530 [D loss: 0.134887] [G loss: -4.515194]
29531 [D loss: -1.141916] [G loss: -4.346462]
29532 [D loss: -0.832934] [G loss: -5.155754]
29533 [D loss: 0.145822] [G loss: -4.721282]
29534 [D loss: 0.809340] [G loss: -4.73

29693 [D loss: -0.289196] [G loss: -3.965955]
29694 [D loss: 0.002039] [G loss: -4.243864]
29695 [D loss: 0.457384] [G loss: -4.542655]
29696 [D loss: -0.264840] [G loss: -4.253328]
29697 [D loss: -0.508036] [G loss: -4.130205]
29698 [D loss: -0.434214] [G loss: -4.325177]
29699 [D loss: -0.074676] [G loss: -3.936205]
29700 [D loss: -0.952917] [G loss: -4.591327]
29701 [D loss: -0.382147] [G loss: -4.561290]
29702 [D loss: 0.059045] [G loss: -4.097126]
29703 [D loss: 0.244974] [G loss: -4.007594]
29704 [D loss: -0.171059] [G loss: -3.929332]
29705 [D loss: 0.646371] [G loss: -4.364903]
29706 [D loss: -0.170615] [G loss: -4.421051]
29707 [D loss: 0.214670] [G loss: -3.989960]
29708 [D loss: -0.478051] [G loss: -4.702802]
29709 [D loss: -0.650087] [G loss: -4.439977]
29710 [D loss: 0.147240] [G loss: -4.431970]
29711 [D loss: -0.885363] [G loss: -4.163805]
29712 [D loss: -1.239794] [G loss: -4.026624]
29713 [D loss: -0.361616] [G loss: -4.628164]
29714 [D loss: -1.238496] [G loss: -4.362

29873 [D loss: 0.297186] [G loss: -4.801945]
29874 [D loss: -0.057862] [G loss: -4.975646]
29875 [D loss: -1.004461] [G loss: -4.207335]
29876 [D loss: 0.209568] [G loss: -3.965784]
29877 [D loss: -0.124162] [G loss: -4.922602]
29878 [D loss: -0.267452] [G loss: -4.558815]
29879 [D loss: -0.313660] [G loss: -4.988290]
29880 [D loss: -0.487482] [G loss: -4.295330]
29881 [D loss: -0.330022] [G loss: -4.513729]
29882 [D loss: -0.332580] [G loss: -4.690662]
29883 [D loss: -0.218798] [G loss: -4.587182]
29884 [D loss: 0.104378] [G loss: -5.143176]
29885 [D loss: 0.267950] [G loss: -4.560204]
29886 [D loss: 0.583966] [G loss: -4.475986]
29887 [D loss: -0.142267] [G loss: -5.028164]
29888 [D loss: -0.545916] [G loss: -4.843975]
29889 [D loss: -0.772059] [G loss: -4.411725]
29890 [D loss: -0.836053] [G loss: -4.634732]
29891 [D loss: -0.475336] [G loss: -4.562161]
29892 [D loss: 0.088807] [G loss: -4.617852]
29893 [D loss: 0.346557] [G loss: -5.291692]
29894 [D loss: -0.511328] [G loss: -4.762

In [1]:
def generate_images(cnt):
    c=1
    while (c<=cnt):
        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = wgan.generator.predict(noise)
        gen_imgs = 0.5*gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\WGAN_Improved Images\wgan_improved_generated\wgan_imp_{}.jpg".format(c), bbox_inches = 'tight', pad_inches = -0.0005)
        print("Images {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)

Using TensorFlow backend.
W0928 09:09:31.149992 23192 deprecation.py:323] From C:\Users\KIIT\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\compat\v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    wgan = WGANGP()
    wgan.train(epochs=30000, batch_size=32, sample_interval=100)

W0928 09:09:31.248017 23192 deprecation.py:506] From C:\Users\KIIT\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)      

c:\users\kiit\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\kiit\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 7.543037] [G loss: 0.329998]


c:\users\kiit\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

29147 [D loss: 0.511652] [G loss: -5.769738]
29148 [D loss: 0.598412] [G loss: -5.351381]
29149 [D loss: -0.157558] [G loss: -5.539743]
29150 [D loss: -1.031176] [G loss: -5.867243]
29151 [D loss: 0.140980] [G loss: -5.197731]
29152 [D loss: -0.989094] [G loss: -5.751698]
29153 [D loss: 0.268244] [G loss: -5.902461]
29154 [D loss: 0.465518] [G loss: -5.553026]
29155 [D loss: -0.213280] [G loss: -5.132071]
29156 [D loss: -0.629004] [G loss: -5.073639]
29157 [D loss: -0.423710] [G loss: -5.653068]
29158 [D loss: -0.798079] [G loss: -5.961239]
29159 [D loss: 0.261894] [G loss: -4.843339]
29160 [D loss: -0.002335] [G loss: -5.859062]
29161 [D loss: 0.318018] [G loss: -5.565143]
29162 [D loss: 0.102515] [G loss: -5.455722]
29163 [D loss: -0.874678] [G loss: -5.513006]
29164 [D loss: -0.558558] [G loss: -5.687809]
29165 [D loss: -0.905250] [G loss: -5.717648]
29166 [D loss: 0.045863] [G loss: -5.884805]
29167 [D loss: -0.393340] [G loss: -5.690913]
29168 [D loss: 0.173806] [G loss: -5.133769

29327 [D loss: 0.232508] [G loss: -5.811637]
29328 [D loss: 0.349782] [G loss: -6.076180]
29329 [D loss: -0.999190] [G loss: -5.488002]
29330 [D loss: -0.355188] [G loss: -5.752138]
29331 [D loss: 0.432353] [G loss: -5.868382]
29332 [D loss: -0.345112] [G loss: -5.222342]
29333 [D loss: 0.401012] [G loss: -5.819486]
29334 [D loss: -0.474260] [G loss: -5.941016]
29335 [D loss: 0.151329] [G loss: -5.994075]
29336 [D loss: -0.296387] [G loss: -5.683578]
29337 [D loss: -0.152813] [G loss: -5.542997]
29338 [D loss: -0.769638] [G loss: -5.097548]
29339 [D loss: -0.016136] [G loss: -5.774771]
29340 [D loss: -0.323314] [G loss: -5.058739]
29341 [D loss: -0.182225] [G loss: -4.768310]
29342 [D loss: 0.545162] [G loss: -5.597394]
29343 [D loss: 0.351022] [G loss: -4.719403]
29344 [D loss: 0.014557] [G loss: -4.909326]
29345 [D loss: 0.051202] [G loss: -5.728593]
29346 [D loss: 0.301390] [G loss: -5.530101]
29347 [D loss: 0.488753] [G loss: -5.270985]
29348 [D loss: -0.397372] [G loss: -5.316225]

29507 [D loss: -0.332630] [G loss: -5.439668]
29508 [D loss: 0.169005] [G loss: -5.107434]
29509 [D loss: -0.335065] [G loss: -4.502538]
29510 [D loss: -0.052669] [G loss: -5.051174]
29511 [D loss: 0.003581] [G loss: -5.008378]
29512 [D loss: -0.797309] [G loss: -5.246051]
29513 [D loss: -0.858526] [G loss: -4.826361]
29514 [D loss: -0.053247] [G loss: -5.296359]
29515 [D loss: 0.157392] [G loss: -5.858953]
29516 [D loss: -0.036728] [G loss: -5.351317]
29517 [D loss: 0.460826] [G loss: -4.507956]
29518 [D loss: -0.586133] [G loss: -4.573533]
29519 [D loss: -0.603524] [G loss: -5.275280]
29520 [D loss: 0.790579] [G loss: -5.297184]
29521 [D loss: -0.190514] [G loss: -5.276599]
29522 [D loss: -0.180601] [G loss: -5.085187]
29523 [D loss: 0.313353] [G loss: -4.776444]
29524 [D loss: -0.193267] [G loss: -5.726407]
29525 [D loss: -0.179444] [G loss: -5.087663]
29526 [D loss: -0.135890] [G loss: -5.216213]
29527 [D loss: -0.207968] [G loss: -5.536100]
29528 [D loss: -0.895036] [G loss: -5.39

29687 [D loss: -0.443906] [G loss: -5.644393]
29688 [D loss: -0.037762] [G loss: -5.842350]
29689 [D loss: 0.911052] [G loss: -5.097161]
29690 [D loss: 0.369998] [G loss: -5.332660]
29691 [D loss: -0.841763] [G loss: -5.390950]
29692 [D loss: -0.275622] [G loss: -6.069163]
29693 [D loss: 0.535270] [G loss: -5.044912]
29694 [D loss: -0.222233] [G loss: -4.922602]
29695 [D loss: -0.438469] [G loss: -5.007130]
29696 [D loss: 0.824543] [G loss: -5.511449]
29697 [D loss: -0.493689] [G loss: -5.258594]
29698 [D loss: -0.321675] [G loss: -5.026879]
29699 [D loss: 0.102758] [G loss: -4.705760]
29700 [D loss: -0.415540] [G loss: -5.203863]
29701 [D loss: 0.365043] [G loss: -4.824789]
29702 [D loss: -0.868838] [G loss: -4.731061]
29703 [D loss: 0.610583] [G loss: -4.828987]
29704 [D loss: 0.272613] [G loss: -5.543443]
29705 [D loss: -0.312844] [G loss: -5.432388]
29706 [D loss: -0.835344] [G loss: -5.527892]
29707 [D loss: -0.524694] [G loss: -5.402573]
29708 [D loss: 0.170863] [G loss: -4.63769

29867 [D loss: 1.008441] [G loss: -5.297277]
29868 [D loss: 0.028940] [G loss: -5.726686]
29869 [D loss: 0.030576] [G loss: -5.210694]
29870 [D loss: 0.162746] [G loss: -5.361354]
29871 [D loss: -0.229209] [G loss: -5.512609]
29872 [D loss: 0.414269] [G loss: -6.007500]
29873 [D loss: -0.191843] [G loss: -5.890107]
29874 [D loss: -0.675263] [G loss: -6.145341]
29875 [D loss: -0.209460] [G loss: -5.314255]
29876 [D loss: -0.494116] [G loss: -5.320622]
29877 [D loss: -0.961734] [G loss: -5.497809]
29878 [D loss: -0.611546] [G loss: -5.327806]
29879 [D loss: -0.243947] [G loss: -5.978350]
29880 [D loss: -0.015158] [G loss: -5.525102]
29881 [D loss: -0.207197] [G loss: -5.709258]
29882 [D loss: -0.590519] [G loss: -5.079875]
29883 [D loss: -0.146318] [G loss: -5.283760]
29884 [D loss: 0.770333] [G loss: -5.142891]
29885 [D loss: 0.511092] [G loss: -5.303868]
29886 [D loss: -0.506056] [G loss: -4.976663]
29887 [D loss: -0.160499] [G loss: -5.888039]
29888 [D loss: -0.039370] [G loss: -5.695